In [1]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas_datareader import data
from dask.distributed import Client
from dask.distributed import as_completed
import openpyxl
import xlsxwriter
import pandas_datareader.data as web

In [2]:
end = datetime.today()
start = end - relativedelta(years=2)

In [3]:
end.strftime('%Y-%m-%d')

'2022-06-05'

In [4]:
etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

FRED_Indicators = [
"ASPUS",\
"AWHAETP",\
"B432RG3A086NBEA",\
"BAA10Y",\
"BACDINA066MNFRBNY",\
"BACTSAMFRBDAL",\
"BAMLC0A0CM",\
"BAMLCC0A1AAATRIV",\
"BAMLCC0A4BBBTRIV",\
"BAMLH0A3HYC",\
"BAMLHYH0A0HYM2TRIV",\
"BOGMBASE",\
"BOGZ1FA105015103Q",\
"BOGZ1FA145050005Q",\
"BOGZ1FA205050005Q",\
"BOGZ1FA315050005Q",\
"BOGZ1FA385050005Q",\
"BOGZ1FL105015105Q",\
"BUSAPPWNSACA",\
"CASTHPI",\
"CES0500000003",\
"CES4348400001",\
"CFNAI",\
"CFNAIDIFF",\
"CFSBCACTIVITYMFG",\
"CILACBQ158SBOG",\
"CIVPART",\
"COMREPUSQ159N",\
"CP",\
"CPALTT01USQ657N",\
"CPIAUCSL",\
"CSCICP03USM665S",\
"CSUSHPINSA",\
"CUSR0000SEHA",\
"CUUR0000SEHA",\
"DALLCIACBEP",\
"DCOILBRENTEU",\
"DCOILWTICO",\
"DEXCHUS",\
"DEXUSUK",\
"DFF",\
"DFII10",\
"DGORDER",\
"DGS1",\
"DGS10",\
"DGS1MO",\
"DGS2",\
"DGS30",\
"DGS3MO",\
"DGS5",\
"DRBLACBS",\
"DRCCLACBS",\
"DRSFRMACBS",\
"DTB3",\
"DTWEXBGS",\
"EMRATIO",\
"ERENTUSQ176N",\
"ETOTALUSQ176N",\
"FEDFUNDS",\
"FGCCSAQ027S",\
"FPCPITOTLZGUSA",\
"GASREGW",\
"GDPC1",\
"GFDEBTN",\
"GFDEGDQ188S",\
"GOLDAMGBD228NLBM",\
"GVZCLS",\
"HDTGPDUSQ163N",\
"HOSMEDUSM052N",\
"IC4WSA",\
"ICSA",\
"IIPUSNETIQ",\
"INDPRO",\
"INTDSRUSM193N",\
"IRLTLT01USM156N",\
"KCFSI",\
"LES1252881600Q",\
"LEU0252918500Q",\
"LFWA64TTUSM647S",\
"LRUN64TTUSQ156S",\
"LXXRCSA",\
"M1",\
"M1V",\
"M2V",\
"MABMM301USM189S",\
"MANMM101USA189S",\
"MEFAINUSA672N",\
"MEHOINUSA672N",\
"MEPAINUSA672N",\
"MICH",\
"MORTGAGE30US",\
"MPRIME",\
"MSPUS",\
"MZMSL",\
"NFCI",\
"NROU",\
"NYSTHPI",\
"PAYEMS",\
"PERMIT",\
"POP",\
"POPTOTUSA647NWDB",\
"PPIACO",\
"PRFI",\
"PSAVERT",\
"Q10096USQ144NNBR",\
"RECPROUSM156N",\
"RSAHORUSQ156S",\
"SPCS20RSA",\
"STLFSI2",\
"T10Y2Y",\
"T10Y3M",\
"T10YIE",\
"T5YIE",\
"T5YIFR",\
"TB3MS",\
"TCU",\
"TDSP",\
"TEDRATE",\
"TOTALSA",\
"TREAST",\
"TTLHH",\
"TWEXB",\
"UMCSENT",\
"U6RATE",\
"UNRATE",\
"USPHCI",\
"USREC",\
"USROA",\
"USROE",\
"USSLIND",\
"USSTHPI",\
"VIXCLS",\
"VXVCLS",\
"WDFUELLA",\
"WGS3MO",\
"WILLLRGCAPVAL",\
"WILLMICROCAPPR",\
"WPU0911",\
"WTB3MS"    
]

etf_bonds = [*etf_bonds,*etf_muni_bonds,*etf_treasuries]

In [5]:
SP500ticker_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP600ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1]
SP400ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0]


In [6]:
sp500 = SP500ticker_list['Symbol']
sp500 = [w.replace('.', '-') for w in sp500]
sp600 = SP600ticker_list['Ticker symbol']
sp600 = [w.replace('.', '-') for w in sp600]
sp400 = SP400ticker_list['Ticker symbol']
sp400 = [w.replace('.', '-') for w in sp400]


In [7]:
len(sp500)+len(sp600)+len(sp400)

1505

In [8]:
print(len(sp500))
SP1500 = list()
SP1500.extend(sp500)
sp600 = list(set(sp600).difference(SP1500))
print(len(sp600))
SP1500.extend(sp600)
sp400 = list(set(sp400).difference(SP1500))
print(len(sp400))
SP1500.extend(set(sp400).difference(SP1500))

504
601
400


In [9]:
sp1500_index_df = pd.DataFrame()

lists = ["sp500","sp600","sp400"]

for l in lists:
    if(l=='sp500'):
        df = pd.DataFrame(sp500,columns=['Symbol'])
        print(len(df))
        #range_ = 
        #print(range_)
        df['Market Index'] = pd.Series(["SP500" for x in range(0,len(df.index),1)])
    elif(l=='sp600'):
        df = pd.DataFrame(np.array(sp600),columns=['Symbol'])        
        print(len(df))        
        df['Market Index'] = pd.Series(["SP600" for x in range(len(df.index))])
        df.index = range(len(sp500),(len(sp500)+len(df.index)),1)        
    else:
        df = pd.DataFrame(np.array(sp400),columns=['Symbol'])
        print(len(df))
        df['Market Index'] = pd.Series(["SP400" for x in range(len(df.index))])
        df.index = range((len(sp500)+len(sp600)),(len(sp500)+len(sp600)+len(df.index)),1)
    sp1500_index_df = pd.concat([sp1500_index_df,df],axis=0)

504
601
400


In [10]:
len(sp1500_index_df)-len(SP1500)

0

In [11]:
def Fred_Data(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    temp = web.DataReader(str(name), 'fred', start_, end_)
    temp.index = pd.to_datetime(temp.index)
    temp.columns = ['Value']
    temp['Symbol'] = name
    #temp = temp.resample(frequency).mean().dropna()
    return(temp)

def dl(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    subset = yf.download(name, start=start_, end=end_, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset['Symbol'] = name

    return (subset)
    
def dl2(assets):

    yahoo_financials = YahooFinancials(assets)

    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)

In [12]:
def getStock(npa):
    #position = npa[0]
    symbol = npa[0]
    start_=npa[1]
    end_=npa[2]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    data_ = yf.download(symbol, start=start_,end=end_)
    #data_['Sector'] = tickerdata.info['sector']
    #data_['Symbol'] = symbol
    #data_['Market Cap'] = data.get_quote_yahoo(names[n])['marketCap']
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [13]:
def getTickerData(npa):
    symbol = npa[0]
    
    stock_data = yf.Ticker(symbol) 
    return([symbol,stock_data])

In [14]:
def getMarketData(npa):
    symbol = npa[0]

    #start_=npa[2]
    #end_=npa[3]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    #data_ = data.get_quote_yahoo(symbol)['marketCap']
    data_ = data.get_quote_yahoo(symbol)
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [15]:
def getSector(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    return([symbol,tickerdata.info['sector']])

In [16]:
def derive_price_supply_trends(npa_):
    name = npa_[0]
    data = npa_[1]
    data_ = data
    data_['Symbol'] = name
    """
    if(np.sum(duplicates==names_prices[n])>0):
        #potential duplicate
        if((np.sum((np.unique(prices_df['Symbol']))==n))>0):
            pass
        else:
            prices_df = pd.concat([prices_df,data_],axis=0)            
    else:
    """
    
    data_['trailing_1yr_max'] = (pd.DataFrame(data_)['High']).rolling(252).max()
    data_['MIN_Lookback_One_Year'] = data_['trailing_1yr_max'].rolling(252).min()
    data_['MIN_Lookback_6_Months'] = data_['trailing_1yr_max'].rolling(126).min()
    data_['MIN_Lookback_2_Months'] = data_['trailing_1yr_max'].rolling(42).min()
    data_['MIN_Lookback_1_Months'] = data_['trailing_1yr_max'].rolling(21).min()

    data_['trailing_1yr_min'] = (pd.DataFrame(data_)['Low']).rolling(252).min()
    
    data_['supply_trend_1yr'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(252).min()
    
    data_['supply_trend_180d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(126).min()
    
    data_['supply_trend_90d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(63).min()
    
    data_['supply_trend_60d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(42).min()
    
    data_['supply_trend_30d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(21).min()

    data_['trailing_60d_max'] = (pd.DataFrame(data_)['High']).rolling(42).max()
    data_['trailing_60d_min'] = (pd.DataFrame(data_)['Low']).rolling(42).min()
    
    data_['trailing_30d_max'] = (pd.DataFrame(data_)['High']).rolling(21).max()
    data_['trailing_30d_min'] = (pd.DataFrame(data_)['Low']).rolling(21).min()

    data_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
    data_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()
    
    data_['risk_per_share'] = data_['trailing_risk_40d_max']-data_['trailing_risk_40d_min']
    
    data_['30d_vol'] = (pd.DataFrame(data_)['Volume']).rolling(21).sum()
    
    data_['momentum_1yr'] = (data_['Adj Close']-data_['MIN_Lookback_One_Year']) + (data_['Adj Close']-data_['MIN_Lookback_6_Months']) + (data_['Adj Close']-data_['MIN_Lookback_2_Months']) + (data_['Adj Close']-data_['MIN_Lookback_1_Months'])
    
    data_['risk_trend_factor'] = data_['momentum_1yr']/data_['risk_per_share']
    
    return(data_)

In [17]:
def extractTickerValues(npa):
    n_ = npa[0]
    r_ = npa[1]
    #print(r_)
    
    subset = pd.DataFrame([r_.info])
    #print(subset)
    
    if(np.sum(np.where(subset.columns=='beta'))==0):
        b_ = 'error'
    else:
        b_ = subset['beta'].values[0]
        
    if(np.sum(np.where(subset.columns=='returnOnAssets'))==0):
        roa_ = 'error'
    else:
        roa_ = subset['returnOnAssets'].values[0]
                
    if(np.sum(np.where(subset.columns=='returnOnEquity'))==0):
        roe_ = 'error'
    else:
        roe_ = subset['returnOnEquity'].values[0]
        
    if(np.sum(np.where(subset.columns=='debtToEquity'))==0):
        debtToEquity = 'error'
    else:
        debtToEquity = subset['debtToEquity'].values[0]
        
    if(np.sum(np.where(subset.columns=='quickRatio'))==0):
        quickRatio = 'error'
    else:
        quickRatio = subset['quickRatio'].values[0]
        
    if(np.sum(np.where(subset.columns=='dividendRate'))==0):
        dividendRate = 'error'
    else:        
        dividendRate = subset['dividendRate'].values[0]    

    data_ = pd.DataFrame([[b_,roa_,roe_,quickRatio,debtToEquity,dividendRate]],columns=['beta','roa','roe','debtToEquity','quickRatio','dividendDate'],index=['n_'])
    
    return([n_,data_])

In [18]:
def calculateMetrics(data_):
    data_df = data_
    n_ = data_df['Symbol'].values[-1]
    
    latest_p = pd.DataFrame(data_df)['Adj Close'][-1]
    oneYearDate = pd.DataFrame(data_df)['Adj Close'][[-1]].index[0] - relativedelta(years=1)
    
    #separate subset for past year.
    past_year = data_df[data_df.index>=oneYearDate.strftime('%Y-%m-%d')]
    
    #np.max(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['High'])
    max_2y = pd.DataFrame(data_df)['High'].rolling(504).max()[-1]
    min_2y = pd.DataFrame(data_df)['Low'].rolling(504).min()[-1]
    
    #quantiles_close_2y = data_df['Adj Close'].rolling(504).quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])[-1]
    quantiles_close_2y = data_df['Adj Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    
    std_2y = pd.DataFrame(data_df)['Adj Close'].rolling(504).std()[-1]
    mean_2y = pd.DataFrame(data_df)['Adj Close'].rolling(504).mean()[-1]
    
    max_1y = past_year['High'].rolling(252).max()[-1]
    min_1y = past_year['Low'].rolling(252).min()[-1]
    
    #quantiles_close_1y = data_df['Adj Close'].rolling(252).quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])[-1]
    quantiles_close_1y = past_year['Adj Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    
    std_1y = past_year['Adj Close'].rolling(252).std()[-1]
    mean_1y = past_year['Adj Close'].rolling(252).mean()[-1]
    
    risk_trend = data_df['risk_per_share'][-1]
    
    supply_trend_1yr = data_df['supply_trend_1yr'][-1]
    supply_trend_180d = data_df['supply_trend_180d'][-1]
    supply_trend_90d = data_df['supply_trend_90d'][-1]
    supply_trend_60d = data_df['supply_trend_60d'][-1]
    supply_trend_30d = data_df['supply_trend_30d'][-1]
    
    MIN_Lookback_One_Year = data_df['MIN_Lookback_One_Year'][-1]
    MIN_Lookback_6_Months = data_df['MIN_Lookback_6_Months'][-1]
    MIN_Lookback_2_Months = data_df['MIN_Lookback_2_Months'][-1]
    MIN_Lookback_1_Months = data_df['MIN_Lookback_1_Months'][-1]
    
    #vol_30d_2yr = data_df['30d_vol'].rolling(504).mean()[-1]
    vol_30d_2yr = np.mean(data_df['30d_vol'])
    
    pct_2yr = (latest_p - min_2y)/(max_2y-min_2y)
    pct_1yr = (latest_p - min_1y)/(max_1y-min_1y)
    
    return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,pct_2yr,MIN_Lookback_One_Year, MIN_Lookback_6_Months, MIN_Lookback_2_Months, MIN_Lookback_1_Months])

In [19]:
#test
"""
dates = []
for d in pd.read_csv('sample.csv')['Date']:
    dates.append(datetime.strptime(d, '%d-%b-%y'))

temp = pd.read_csv('sample.csv',thousands=",")
temp.index = dates
temp = temp.iloc[:,1:]
temp.columns = ['Open','High','Low','Adj Close','Volume']
#temp['Close'] = temp['Close'].astype(float)
temp.sort_index(inplace=True)
temp

derive_price_supply_trends(['test',temp]).sort_index(ascending=False)
calculateMetrics(temp)
"""

'\ndates = []\nfor d in pd.read_csv(\'sample.csv\')[\'Date\']:\n    dates.append(datetime.strptime(d, \'%d-%b-%y\'))\n\ntemp = pd.read_csv(\'sample.csv\',thousands=",")\ntemp.index = dates\ntemp = temp.iloc[:,1:]\ntemp.columns = [\'Open\',\'High\',\'Low\',\'Adj Close\',\'Volume\']\n#temp[\'Close\'] = temp[\'Close\'].astype(float)\ntemp.sort_index(inplace=True)\ntemp\n\nderive_price_supply_trends([\'test\',temp]).sort_index(ascending=False)\ncalculateMetrics(temp)\n'

In [20]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_prices = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_prices.append("error")
    else:
        results_prices.append(f.result()) 
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [21]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getSector(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getSector, npa)
results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_sectors.append("error")
    else:
        results_sectors.append(f.result()) 
client.close()


In [22]:
len(np.unique([r[0] for r in results_sectors]))

1087

In [23]:
#Drop Down screen metrics
#by sector
#further subdivide by market
#https://stackoverflow.com/questions/64919511/dateformatter-is-bringing-1970-as-year-not-the-original-year-in-the-dataset


In [24]:
sectors_df = pd.DataFrame()

for r in range(0,len(results_sectors)):
    r_ = results_sectors[r]
    if(r_=='error'):
        pass#sectors_df = pd.concat([sectors_df,pd.DataFrame([np.nan],index=[i_],columns=['Sector'])],axis=0)
    else:
        s_ = np.array(r_)[1]
        i_ = np.array(r_)[0]
        sectors_df = pd.concat([sectors_df,pd.DataFrame([s_],index=[i_],columns=['Sector'])],axis=0)

missing_sectors = list(set(SP1500).difference([r[0] for r in results_sectors]))

for r in range(0,len(missing_sectors)):
    r_ = missing_sectors[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    sectors_df = pd.concat([sectors_df,pd.DataFrame(['error'],index=[r_],columns=['Sector'])],axis=0)

In [25]:
#sectors_df

In [26]:
#mktcap_df

In [27]:
#np.sum(sectors_df.index.duplicated())

In [28]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [29]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getMarketCap(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getMarketData, npa)
results_mkt_data = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_mkt_data.append("error")
    else:
        results_mkt_data.append(f.result()) 
client.restart()
client.close()


In [30]:
#results_mkt_data = results_mktcp

In [31]:
results_mkt_data[0][1].columns

Index(['language', 'region', 'quoteType', 'triggerable', 'quoteSourceName',
       'postMarketChangePercent', 'postMarketTime', 'postMarketPrice',
       'postMarketChange', 'regularMarketChangePercent',
       'regularMarketDayRange', 'regularMarketPreviousClose', 'bid', 'ask',
       'bidSize', 'askSize', 'messageBoardId', 'epsTrailingTwelveMonths',
       'epsForward', 'marketCap', 'forwardPE', 'priceToBook', 'sourceInterval',
       'exchangeTimezoneName', 'exchangeTimezoneShortName',
       'gmtOffSetMilliseconds', 'currency', 'esgPopulated', 'tradeable',
       'epsCurrentYear', 'priceEpsCurrentYear', 'sharesOutstanding',
       'bookValue', 'fiftyDayAverage', 'fiftyDayAverageChange',
       'fiftyDayAverageChangePercent', 'twoHundredDayAverage',
       'twoHundredDayAverageChange', 'twoHundredDayAverageChangePercent',
       'priceHint', 'fullExchangeName', 'longName', 'financialCurrency',
       'averageDailyVolume3Month', 'averageDailyVolume10Day',
       'fiftyTwoWeekLowChang

In [32]:
#results_mktcp[10][1].columns

In [33]:
#np.array([r[0] for r in results_mktcp])
#names_mktcap = [n[0] for n in results_mktcp]

mktcap_df = pd.DataFrame()

for r in range(0,len(results_mkt_data)):
    r_ = results_mkt_data[r]
    if(r_=='error'):
        pass#sectors_df = pd.concat([sectors_df,pd.DataFrame([np.nan],index=[i_],columns=['Sector'])],axis=0)
    else:
        if(np.sum(np.where(np.array(r_[1].columns)=='marketCap'))==0):
            pass
        else:
            m_ = r_[1]['marketCap'][0]
            #print(s_[0])
            i_ = np.array(r_)[0]
            mktcap_df = pd.concat([mktcap_df,pd.DataFrame([m_],index=[i_],columns=['Market Cap'])],axis=0)



C:\Users\User\AppData\Local\Temp\ipykernel_25016\1182025761.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  i_ = np.array(r_)[0]


In [34]:
missing_mktcaps = list(set(SP1500).difference([n for n in mktcap_df.index]))

for r in range(0,len(missing_mktcaps)):
    r_ = missing_mktcaps[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    mktcap_df = pd.concat([mktcap_df,pd.DataFrame([np.nan],index=[r_],columns=['Market Cap'])],axis=0)

In [35]:
mktcap_df

,Market Cap
ETD,5.723916e+08
PK,4.363038e+09
TOL,6.021610e+09
FORM,3.149412e+09
WTFC,5.085621e+09
...,...
JW-A,NaN
ZIMV,NaN
APA,NaN
OMCL,NaN


In [36]:
pe_trailing_df = pd.DataFrame()
    
npa = []
for r in range(0,len(results_mkt_data)):
    r_ = results_mkt_data[r]
    if(r_=='error'):
        pass
    else:
        if(np.sum(np.where(r_[1].columns=='trailingPE'))==0):
            pass
        else:
            pe_trailing_df = pd.concat([pe_trailing_df,pd.DataFrame(r_[1]['trailingPE'])],axis=0)
            
missing_pe_trailing = list(set(SP1500).difference([n for n in pe_trailing_df.index]))

for r in range(0,len(missing_pe_trailing)):
    r_ = missing_pe_trailing[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    pe_trailing_df = pd.concat([pe_trailing_df,pd.DataFrame([np.nan],index=[r_],columns=['trailingPE'])],axis=0)
            


In [37]:
pe_trailing_df

,trailingPE
ETD,8.127689
TOL,7.546003
FORM,41.515465
WTFC,11.705813
CVCO,18.314398
...,...
PRGO,NaN
ATNI,NaN
CENX,NaN
APA,NaN


In [38]:
pe_forward_df = pd.DataFrame()
    
npa = []
for r in range(0,len(results_mkt_data)):
    r_ = results_mkt_data[r]
    if(r_=='error'):
        pass
    else:
        if(np.sum(np.where(r_[1].columns=='forwardPE'))==0):
            pass
        else:
            pe_forward_df = pd.concat([pe_forward_df,pd.DataFrame(r_[1]['forwardPE'])],axis=0)
            
missing_pe_forward = list(set(SP1500).difference([n for n in pe_forward_df.index]))

for r in range(0,len(missing_pe_forward)):
    r_ = missing_pe_forward[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    pe_forward_df = pd.concat([pe_forward_df,pd.DataFrame([np.nan],index=[r_],columns=['forwardPE'])],axis=0)
            


In [39]:
pe_forward_df

,forwardPE
ETD,7.707483
PK,307.500030
TOL,4.265132
FORM,22.751413
WTFC,14.893156
...,...
CRVL,NaN
APA,NaN
OMCL,NaN
XHR,NaN


In [40]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getMarketCap(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getTickerData, npa)

results_ticker_data = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_data.append("error")
    else:
        results_ticker_data.append(f.result()) 
client.restart()
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [41]:
print(results_ticker_data[0][1].info)
print(results_ticker_data[0][1].quarterly_balance_sheet)
print(results_ticker_data[0][1].quarterly_cashflow)
#print(results_ticker_data[0][1].quarterly_dividends)
print(results_ticker_data[0][1].quarterly_earnings)
print(results_ticker_data[0][1].quarterly_financials)


{'zip': '55144-1000', 'sector': 'Industrials', 'fullTimeEmployees': 95000, 'longBusinessSummary': '3M Company operates as a diversified technology company worldwide. It operates through four segments: Safety and Industrial; Transportation and Electronics; Health Care; and Consumer. The Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; closure systems for personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection solutions; and natural and color-coated mineral granules for shingles. The Transportation and Electronics segment provides ceramic solutions; attachment tapes, films, sound, and temperature management for transportation vehicles; premium large format graphic films for advertising and f

In [42]:
results_ticker_data[0][1].info

{'zip': '55144-1000',
 'sector': 'Industrials',
 'fullTimeEmployees': 95000,
 'longBusinessSummary': '3M Company operates as a diversified technology company worldwide. It operates through four segments: Safety and Industrial; Transportation and Electronics; Health Care; and Consumer. The Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; closure systems for personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection solutions; and natural and color-coated mineral granules for shingles. The Transportation and Electronics segment provides ceramic solutions; attachment tapes, films, sound, and temperature management for transportation vehicles; premium large format graphic films for advertising an

In [43]:
npa = []
results_ticker_names = [r[0] for r in results_ticker_data]
for n in range(0,len(results_ticker_names),1):
    npa.append([results_ticker_names[n],results_ticker_data[n][1]])

In [44]:

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(extractTickerValues, npa)

results_ticker_values = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_values.append("error")
    else:
        results_ticker_values.append(f.result()) 
client.restart()
client.close()



C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [45]:
ticker_values = pd.DataFrame()

for r in results_ticker_values:
    if(r=='error'):
        pass
    else:                
        #print(r[1])
        subset = pd.DataFrame(r[1])
        subset.index = [r[0]]
        #print(subset)
        ticker_values = pd.concat([ticker_values,subset],axis=0)
        #ticker_values = pd.concat([ticker_values,pd.DataFrame(r[1],index=[r[0]])],axis=0)

In [46]:
market_data = pd.concat([mktcap_df,ticker_values,sectors_df.loc[~sectors_df.index.duplicated(), :].reindex(mktcap_df.loc[~mktcap_df.index.duplicated(), :].index)],axis=1)

In [47]:
market_data.sort_values(by='Market Cap',ascending=False)

,Market Cap,beta,roa,roe,debtToEquity,quickRatio,dividendDate,Sector
AAPL,2.385163e+12,1.194642,0.21695,1.49271,0.76,178.016,0.92,Technology
MSFT,2.027305e+12,0.93616,0.15674,0.48721,1.773,47.863,2.48,Technology
GOOG,1.520775e+12,1.132142,0.15049,0.308,2.738,11.26,None,Communication Services
GOOGL,1.520649e+12,1.132142,0.15049,0.308,2.738,11.26,None,Communication Services
AMZN,1.240991e+12,1.231696,0.03395,0.18046,0.707,109.9,None,Consumer Cyclical
...,...,...,...,...,...,...,...,...
JW-A,NaN,error,error,error,error,error,error,error
ZIMV,NaN,None,-0.0242,-0.06479,1.433,3.989,None,Healthcare
APA,NaN,4.135714,0.17293,4.69732,0.882,704.108,0.5,Energy
OMCL,NaN,error,error,error,error,error,error,error


In [48]:
market_data

,Market Cap,beta,roa,roe,debtToEquity,quickRatio,dividendDate,Sector
ETD,5.723916e+08,1.043303,0.10388,0.24286,0.504,30.825,1.28,Consumer Cyclical
PK,4.363038e+09,1.927678,-0.00273,-0.07093,3.725,114.842,0.01,Real Estate
TOL,6.021610e+09,1.489285,0.06712,0.18989,0.308,63.714,0.71,Consumer Cyclical
FORM,3.149412e+09,error,error,error,error,error,error,Technology
WTFC,5.085621e+09,error,error,error,error,error,error,Financial Services
...,...,...,...,...,...,...,...,...
JW-A,NaN,error,error,error,error,error,error,error
ZIMV,NaN,None,-0.0242,-0.06479,1.433,3.989,None,Healthcare
APA,NaN,4.135714,0.17293,4.69732,0.882,704.108,0.5,Energy
OMCL,NaN,error,error,error,error,error,error,error


In [49]:
#total_mkt_cap = mktcap_df.replace([np.nan], 0).sum()
#mktcap_df['mkt_cap_pct'] = mktcap_df.replace([np.nan], 0)/total_mkt_cap

In [50]:
#[rp[2] for rp in results_prices][0].info['sector']

In [51]:
"""
arr=np.array(names_prices)
pd.Series(arr).value_counts()
set_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]
if(len(set_)==0):
    duplicates = []
    pass
else:
    duplicates = set_.index[0]
    """

'\narr=np.array(names_prices)\npd.Series(arr).value_counts()\nset_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]\nif(len(set_)==0):\n    duplicates = []\n    pass\nelse:\n    duplicates = set_.index[0]\n    '

In [52]:
#len(results)

In [53]:
names_prices = [rp[0] for rp in results_prices]

#def process_df(data_):
npa = []    
for n in range(0,len(names_prices)):
    #print(names_prices[n])
    n_ = names_prices[n]
    npa.append([n_,results_prices[n][1]])
    
prices_df = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(derive_price_supply_trends, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result()) 
        prices_df = pd.concat([f.result(),prices_df],axis=0)
client.close()

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [54]:
#np.unique(np.array(prices_df['Symbol'].values))

In [55]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [56]:
prices_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol',
       'trailing_1yr_max', 'MIN_Lookback_One_Year', 'MIN_Lookback_6_Months',
       'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months', 'trailing_1yr_min',
       'supply_trend_1yr', 'supply_trend_180d', 'supply_trend_90d',
       'supply_trend_60d', 'supply_trend_30d', 'trailing_60d_max',
       'trailing_60d_min', 'trailing_30d_max', 'trailing_30d_min',
       'trailing_risk_40d_max', 'trailing_risk_40d_min', 'risk_per_share',
       '30d_vol', 'momentum_1yr', 'risk_trend_factor'],
      dtype='object')

In [57]:
#[rp[0] for rp in results_prices]

In [58]:
npa = []
for s in names_prices:
    #print(s)
    subset = prices_df.iloc[np.where(prices_df['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    
screenerMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        screenerMetrics.append("error")
    else:
        screenerMetrics.append(f.result()) 
client.close()

missing_screener_metrics = list(set(SP1500).difference([n for n in pd.DataFrame(screenerMetrics).set_index(0).index]))
print(missing_screener_metrics)

for r in range(0,len(missing_screener_metrics)):
    r_ = missing_screener_metrics[r]
    test = np.array([r_,*np.repeat('missing',len(pd.DataFrame(screenerMetrics).set_index(0).iloc[[0]].columns))])
    screenerMetrics.append(test)

pd.DataFrame(screenerMetrics)


['JW-A']


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,THRM,99.0,36.77,15.876002,68.212381,99.0,60.889999,8.633199,78.910357,70.120003,...,6.240002,6.240002,6.240002,3660511.157025,0.242194,0.535915,81.980003,89.589996,99.0,99.0
1,CBRE,111.0,41.029999,20.414821,77.980119,111.0,76.349998,8.112137,94.423254,79.900002,...,1.239998,1.239998,1.239998,34395376.652893,0.102453,0.555524,90.209999,107.879997,111.0,111.0
2,EXR,228.839996,88.879997,40.88964,150.081545,228.839996,154.309998,17.321034,187.30915,178.300003,...,6.440002,6.440002,6.440002,16682716.735537,0.321884,0.638897,157.350006,207.559998,228.839996,228.839996
3,MMM,208.949997,139.740005,16.800345,167.544718,206.809998,139.740005,17.418921,169.728002,145.970001,...,1.490005,0.0,0.0,54812074.793388,0.092888,0.090016,206.809998,206.809998,206.809998,206.809998
4,VIR,141.009995,18.209999,11.287309,39.268036,58.0,18.209999,9.340911,35.543175,25.059999,...,4.369999,4.369999,4.369999,27135600.206612,0.172154,0.055782,58.0,58.0,58.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,TFX,449.380005,256.769989,37.001862,365.386996,428.359985,256.769989,37.914838,347.784176,281.850006,...,16.440002,16.440002,16.440002,5844147.727273,0.146162,0.130211,428.359985,428.359985,428.359985,428.359985
1501,PRLB,286.570007,39.360001,43.919906,99.014068,98.260002,39.360001,14.960858,62.193413,48.75,...,6.23,6.23,6.23,7968685.330579,0.159423,0.037984,98.260002,98.260002,98.260002,98.260002
1502,BANC,22.09,9.0,3.502562,16.626164,22.09,15.4,1.374202,18.915765,18.959999,...,1.009998,1.009998,1.009998,5576172.933884,0.532137,0.760886,21.4,22.09,22.09,22.09
1503,UNVR,34.0,15.57,4.714985,23.428651,34.0,21.559999,2.833027,27.210595,31.58,...,4.0,4.0,4.0,27580397.727273,0.805466,0.868692,27.91,29.77,34.0,34.0


In [59]:
#list(set(SP1500).difference([n for n in pd.DataFrame(screenerMetrics).set_index(0).index]))

In [60]:
index_names = ['SP1500', 'SP500', 'SP600', 'SP400']
indexes = ['SPTM','SPY','SLY','MDYG']
sector_names = ['Communication Services ETF', 'Consumer Discretionary ETF', 'Consumer Staples ETF', 'Energy ETF', 'Financials ETF', 'Health Care ETF', 'Industrials ETF', 'Information Technology ETF', 'Materials ETF', 'Real Estate ETF', 'Utilities ETF']
sectors = ['VOX','VCR','VDC','VDE','VFH','VHT','VIS','VGT','VAW','VNQ','VPU']

In [61]:
npa = []
for i in range(0,len(indexes)):
    npa.append([indexes[i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_indexes = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_indexes.append("error")
    else:
        results_indexes.append(f.result()) 
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [62]:
npa = []
for i in range(0,len(sectors)):
    npa.append([sectors[i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_sectors.append("error")
    else:
        results_sectors.append(f.result()) 
client.close()


In [63]:
#def process_df(data_):
npa = []    
for n in range(0,len(indexes)):
    #print()
    data=results_indexes[n][1]
    data['Symbol'] = indexes[n]
    npa.append([indexes[n],data])
    
prices_indexes = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(derive_price_supply_trends, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result()) 
        prices_indexes = pd.concat([f.result(),prices_indexes],axis=0)
client.close()



In [64]:

#def process_df(data_):
npa = []    
for n in range(0,len(sectors)):
    #print()
    data=results_sectors[n][1]
    data['Symbol'] = sectors[n]
    npa.append([sectors[n],data])
    
prices_sectors = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(derive_price_supply_trends, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result()) 
        prices_sectors = pd.concat([f.result(),prices_sectors],axis=0)
client.close()


In [65]:
npa = []
for s in indexes:
    subset = prices_indexes.iloc[np.where(prices_indexes['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    
indexMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        indexMetrics.append("error")
    else:
        indexMetrics.append(f.result()) 
client.close()


In [66]:
npa = []
for s in sectors:
    subset = prices_sectors.iloc[np.where(prices_sectors['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    
sectorMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        sectorMetrics.append("error")
    else:
        sectorMetrics.append(f.result()) 
client.close()


In [67]:
#market_data.index = [sub.replace('.', '-') for sub in market_data.index]


In [68]:
set(np.array(market_data.index)).difference(pd.DataFrame(screenerMetrics).set_index(0).index)

set()

In [69]:
metrics = pd.concat([pd.DataFrame(screenerMetrics),pd.DataFrame(indexMetrics),pd.DataFrame(sectorMetrics)],axis=0)

In [70]:
#np.min(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['Open'])

In [71]:
#calculateMinMax(npa[np.where(np.array(names_prices)=='GBX')[0][0]])

In [72]:
#pd.DataFrame(screener['risk_trend']).sort_values(by='risk_trend',ascending=False)

In [73]:
prices_df = pd.concat([prices_df,prices_indexes,prices_sectors],axis=0)

In [74]:
npa = []
for f in FRED_Indicators:
    npa.append([f,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])

In [75]:
   
completed_fred = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(Fred_Data, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_fred = pd.concat([completed_fred,f.result()],axis=0)
client.close()

completed_fred_pvt = pd.pivot_table(completed_fred, values=['Value'], index=['DATE'],columns=['Symbol'])
completed_fred_pvt

Value                                                   \
Symbol     ASPUS AWHAETP B432RG3A086NBEA BAA10Y BACDINA066MNFRBNY   
DATE                                                                
2020-06-05   NaN     NaN             NaN   2.86               NaN   
2020-06-06   NaN     NaN             NaN    NaN               NaN   
2020-06-07   NaN     NaN             NaN    NaN               NaN   
2020-06-08   NaN     NaN             NaN   2.81               NaN   
2020-06-09   NaN     NaN             NaN   2.80               NaN   
...          ...     ...             ...    ...               ...   
2022-05-30   NaN     NaN             NaN    NaN               NaN   
2022-05-31   NaN     NaN             NaN   2.18               NaN   
2022-06-01   NaN     NaN             NaN   2.08               NaN   
2022-06-02   NaN     NaN             NaN   2.10               NaN   
2022-06-03   NaN     NaN             NaN    NaN               NaN   

                                                                       \
Symbol     BACTSAMFRBDAL BAMLC0A0CM BAMLCC0A1AAATRIV BAMLCC0A4BBBTRIV   
DATE                                                                    
2020-06-05           NaN       1.60           766.77           938.56   
2020-06-06           NaN        NaN              NaN              NaN   
2020-06-07           NaN        NaN              NaN              NaN   
2020-06-08           NaN       1.56           769.10           943.74   
2020-06-09           NaN       1.59           771.40           945.70   
...                  ...        ...              ...              ...   
2022-05-30           NaN       1.41           680.42           886.65   
2022-05-31           NaN       1.40           676.42           883.75   
2022-06-01           NaN       1.37           674.43           881.33   
2022-06-02           NaN       1.38           673.89           881.84   
2022-06-03           NaN        NaN              NaN              NaN   

                        ...                                              \
Symbol     BAMLH0A3HYC  ... USROE USSTHPI VIXCLS VXVCLS WDFUELLA WGS3MO   
DATE                    ...                                               
2020-06-05       13.21  ...   NaN     NaN  24.52  27.42    1.124   0.15   
2020-06-06         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   
2020-06-07         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   
2020-06-08       13.16  ...   NaN     NaN  25.81  28.08      NaN    NaN   
2020-06-09       13.29  ...   NaN     NaN  27.57  29.67      NaN    NaN   
...                ...  ...   ...     ...    ...    ...      ...    ...   
2022-05-30        9.79  ...   NaN     NaN  26.54    NaN      NaN    NaN   
2022-05-31        9.83  ...   NaN     NaN  26.19  27.99      NaN    NaN   
2022-06-01        9.70  ...   NaN     NaN  25.69  27.72      NaN    NaN   
2022-06-02        9.70  ...   NaN     NaN  24.72  27.06      NaN    NaN   
2022-06-03         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   

                                                        
Symbol     WILLLRGCAPVAL WILLMICROCAPPR WPU0911 WTB3MS  
DATE                                                    
2020-06-05      90778.48       11717.89     NaN   0.15  
2020-06-06           NaN            NaN     NaN    NaN  
2020-06-07           NaN            NaN     NaN    NaN  
2020-06-08      92383.94       12070.97     NaN    NaN  
2020-06-09      90990.35       11984.24     NaN    NaN  
...                  ...            ...     ...    ...  
2022-05-30           NaN            NaN     NaN    NaN  
2022-05-31     123329.51       13744.24     NaN    NaN  
2022-06-01     122355.57       13646.47     NaN    NaN  
2022-06-02     123713.56       13914.28     NaN    NaN  
2022-06-03           NaN            NaN     NaN    NaN  

[729 rows x 130 columns]

In [76]:
npa = []
for e in etf_bonds:
    npa.append([e,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
completed_bonds = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(dl, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_bonds = pd.concat([completed_bonds,f.result()],axis=0)
client.close()

completed_bonds_pvt = pd.pivot_table(completed_bonds, values='Close', index=['Date'],columns=['Symbol'])
completed_bonds_pvt

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Symbol,AGG,AGZ,BIL,GBF,GOVT,IEF,IEI,ILTB,IMTB,ISTB,IUSB,LQD,MEAR,MUB,NEAR,SHV,SHY,SUB,TLT
Date,,,,,,,,,,,,,,,,,,,
2020-06-05,112.658875,116.816986,91.463959,120.032829,26.770966,117.545258,130.427597,68.019234,49.520599,49.601700,51.660980,125.992378,49.648056,110.227272,48.881229,110.389801,85.775223,105.943924,151.403275
2020-06-08,112.784134,116.651855,91.473953,120.441711,26.790386,117.672882,130.496521,68.421707,49.664368,49.572746,51.728138,126.411461,49.657970,110.304329,48.891060,110.389801,85.755417,105.894722,151.985184
2020-06-09,112.928711,116.875259,91.473953,120.509888,26.897194,118.094879,130.762161,68.665070,49.602478,49.582405,51.747341,126.144775,49.667896,110.641457,48.910725,110.389801,85.785133,106.170242,153.711487
2020-06-10,113.391289,117.331749,91.483948,121.084274,27.013712,118.978142,131.244339,69.404518,49.678638,49.669289,51.987217,126.973534,49.687748,110.689644,48.969681,110.389801,85.834740,106.248985,155.990601
2020-06-11,113.198540,117.234634,91.463959,121.055084,27.139950,119.400154,131.283676,69.095634,49.754803,49.553444,51.804897,125.458893,49.588490,110.853401,48.900894,110.409737,85.795059,106.150574,158.938980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-27,103.942810,111.125732,91.428993,109.984833,24.192898,104.279640,120.818497,58.372116,46.006367,48.076820,48.087307,115.013969,49.727005,108.117729,49.417988,110.212982,83.414902,104.721939,118.868523
2022-05-31,103.284004,110.835999,91.418999,109.206009,24.102999,103.191002,120.219002,57.753998,45.696999,47.957001,47.728001,114.166000,49.736996,107.937996,49.408001,110.182999,83.284996,104.612000,116.352997
2022-06-01,102.860001,110.309998,91.419998,108.690002,24.040001,102.610001,119.680000,57.439999,45.480000,47.820000,47.599998,113.500000,49.740002,107.940002,49.389999,110.169998,83.129997,104.660004,116.220001


In [77]:
#return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,,pct_2yr])
metrics.columns = ['Symbol','max_2y','min_2y','std_2y','mean_2y','max_1y','min_1y','std_1y','mean_1y','latest_p','2yr_close_0pct','2yr_close_2pct','2yr_close_9pct','2yr_close_25pct','2yr_close_50pct','2yr_close_75pct','2yr_close_91pct','2yr_close_98pct','2yr_close_100pct','1yr_close_0pct','1yr_close_2pct','1yr_close_9pct','1yr_close_25pct','1yr_close_50pct','2yr_close_75pct','1yr_close_91pct','1yr_close_98pct','1yr_close_100pct','risk_trend','supply_trend_1yr','supply_trend_180d','supply_trend_90d','supply_trend_60d','supply_trend_30d','vol_30d_2yr','pct_1yr','pct_2yr','MIN_Lookback_One_Year', 'MIN_Lookback_6_Months', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months']

In [78]:
completed_bonds

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2020-06-05,49.717535,49.717535,49.641108,49.648056,37000,MEAR
2020-06-08,49.737377,49.737377,49.618271,49.657970,41800,MEAR
2020-06-09,49.717524,49.717524,49.650029,49.667896,51500,MEAR
2020-06-10,49.638117,49.707600,49.638117,49.687748,50400,MEAR
2020-06-11,49.717524,49.717524,49.588490,49.588490,53400,MEAR
...,...,...,...,...,...,...
2022-05-27,48.066833,48.106774,48.036880,48.076820,519900,ISTB
2022-05-31,47.986958,47.986958,47.917064,47.957001,753800,ISTB
2022-06-01,47.939999,47.950001,47.779999,47.820000,493400,ISTB


In [79]:
screener = metrics
a = screener.set_index('Symbol')[screener.set_index('Symbol')['risk_trend'].apply(type) != np.str_].sort_values(by='risk_trend',ascending=False)
b = screener.set_index('Symbol')[screener.set_index('Symbol')['risk_trend'].apply(type) == np.str_]
screener_sorted = pd.concat([a,b],axis=0)


In [80]:
screener_sorted

,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,2yr_close_0pct,...,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,pct_2yr,MIN_Lookback_One_Year,MIN_Lookback_6_Months,MIN_Lookback_2_Months,MIN_Lookback_1_Months
Symbol,,,,,,,,,,,,,,,,,,,,,
AMZN,3773.080078,2025.199951,296.531055,3180.708332,3773.080078,2025.199951,375.036636,3205.775125,2447.0,2082.0,...,339.0,339.0,339.0,82062094.834711,0.241321,0.241321,3554.0,3773.080078,3773.080078,3773.080078
GOOG,3042.0,1347.01001,517.905498,2259.353414,3042.0,2044.160034,201.916485,2704.086196,2291.280029,1359.900024,...,163.380127,163.380127,163.380127,30572394.834711,0.247655,0.557095,2468.0,3037.0,3042.0,3042.0
GOOGL,3030.929932,1351.650024,511.86544,2244.401052,3030.929932,2037.689941,208.932014,2684.163652,2290.820068,1362.540039,...,161.650146,161.650146,161.650146,34995544.628099,0.254853,0.55927,2431.379883,3019.330078,3030.929932,3030.929932
NVR,5982.450195,3050.0,592.039014,4608.963436,5982.450195,3998.0,416.209297,4986.021081,4428.0,3098.629883,...,250.080078,250.080078,250.080078,462794.421488,0.216685,0.469914,5308.47998,5620.529785,5982.450195,5982.450195
AZO,2267.399902,1074.449951,339.630637,1527.495833,2267.399902,1371.140015,212.855434,1819.807737,2058.040039,1095.699951,...,236.910034,236.910034,236.910034,4200658.264463,0.766407,0.824502,1542.300049,1941.359985,2110.0,2267.399902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NYMT,4.93,2.26,0.759132,3.601385,4.93,2.84,0.470891,3.929987,3.11,2.119451,...,0.17,0.17,0.17,70848274.793388,0.129187,0.318352,4.78,4.93,4.93,4.93
TWO,8.15,4.62,0.819822,5.6918,8.15,4.71,0.609238,5.862697,5.33,3.922107,...,0.52,0.52,0.27,99173650.206612,0.180233,0.201133,7.89,8.15,8.15,8.15
GNW,4.76,1.87,0.596799,3.673155,4.61,3.23,0.290099,3.916548,3.99,2.01,...,0.38,0.38,0.14,106843557.231405,0.550725,0.733564,4.61,4.61,4.61,4.61


In [81]:
writer = pd.ExcelWriter('SP1500_screener_multiple.xlsx', engine='xlsxwriter')

prices_df.to_excel(writer, sheet_name='SP1500_Sectors_Indexes_Prices')
#prices_df.drop(['Sector', 'Market Cap'], axis=1).to_excel(writer, sheet_name='SP1500Prices')

sectors_df.to_excel(writer, sheet_name='stock_sectors')

mktcap_df.to_excel(writer, sheet_name='market_caps')

sp1500_index_df.to_excel(writer, sheet_name='SP1500')

pd.DataFrame(sp500).to_excel(writer, sheet_name='SP500')
pd.DataFrame(sp600).to_excel(writer, sheet_name='SP600')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(market_data).to_excel(writer, sheet_name='Stock_Fundamentals')
completed_fred_pvt.to_excel(writer, sheet_name='completed_fred_pvt')
completed_bonds.to_excel(writer, sheet_name='completed_bonds')
completed_bonds_pvt.to_excel(writer, sheet_name='completed_bonds_pvt')
pd.DataFrame(pd.DataFrame(sectors,sector_names)).to_excel(writer, sheet_name='vanguard_sectors')
pd.DataFrame(pd.DataFrame(indexes,index_names)).to_excel(writer, sheet_name='spdr_indexes')
screener_sorted.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500_Sectors_Indexes_Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Sectors', attr_text='stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('market_caps', attr_text='market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Vanguard_Sectors', attr_text='vanguard_sectors!$B$1:$C$'+str(len(sectors)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SPDR_Indexes', attr_text='spdr_indexes!$B$1:$C$'+str(len(indexes)+1))
wb.defined_names.append(new_range)


#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

workbook.close()
"""

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\nworkbook.close()\n"